In [1]:
!pip install psycopg2-binary
import psycopg2

In [2]:
dbname = 'lqsujdux' 
user = 'lqsujdux' 
password =  'w7ih63BvRfpV_Mf9suEjW-6NmOGaTcP-'
host =  'rajje.db.elephantsql.com'

In [3]:
pg_conn = psycopg2.connect(dbname=dbname, user = user, password=password, host=host)
pg_curs = pg_conn.cursor()

In [4]:
import sqlite3
import pandas as pd

In [5]:
sl_conn = sqlite3.connect('titanic.sqlite3')

In [6]:
# transform the titanic csv into a database that i can connect to using sqlite
df = pd.read_csv('https://raw.githubusercontent.com/EvidenceN/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module2-sql-for-analysis/titanic.csv')
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [7]:
def clean(x):
    
    x['Name'] = x['Name'].str.replace("'","`")
    
    return x

In [8]:
new_df = clean(df)

In [9]:
data = new_df.to_sql(name = 'titanic', con=sl_conn)

C:\Users\Samsung\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [10]:
sl_curs = sl_conn.cursor()

In [11]:
sl_curs.execute('SELECT * FROM titanic').fetchone()

(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [12]:
sl_curs.execute('PRAGMA table_info(titanic);').fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'Survived', 'INTEGER', 0, None, 0),
 (2, 'Pclass', 'INTEGER', 0, None, 0),
 (3, 'Name', 'TEXT', 0, None, 0),
 (4, 'Sex', 'TEXT', 0, None, 0),
 (5, 'Age', 'REAL', 0, None, 0),
 (6, 'Siblings/Spouses Aboard', 'INTEGER', 0, None, 0),
 (7, 'Parents/Children Aboard', 'INTEGER', 0, None, 0),
 (8, 'Fare', 'REAL', 0, None, 0)]

In [13]:
create_titanic_table = """
    CREATE TABLE titanic(
    Survived INT,
    Pclass INT,
    Name TEXT,
    Sex TEXT,
    Age REAL,
    Siblings_Spouses_Aboard INT,
    Parents_Children_Aboard INT,
    Fare REAL);
"""

In [14]:
pg_curs.execute(create_titanic_table)

In [15]:
show_tables = """
    SELECT *
    FROM pg_catalog.pg_tables
    WHERE schemaname != 'pg_catalog'
    AND schemaname != 'information_schema';
"""

In [16]:
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'lqsujdux', None, True, False, False, False),
 ('public', 'titanic', 'lqsujdux', None, False, False, False, False),
 ('public',
  'charactercreator_character',
  'lqsujdux',
  None,
  True,
  False,
  False,
  False)]

In [17]:
original_titanic = sl_curs.execute('SELECT * FROM titanic').fetchall()

In [18]:
original_titanic[0]

(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [19]:
str(original_titanic[4][1:])

"(0, 3, 'Mr. William Henry Allen', 'male', 35.0, 0, 0, 8.05)"

In [20]:
for people in original_titanic:
    insert_data = """
        INSERT INTO titanic
        (Survived, Pclass, Name, Sex, Age, Siblings_Spouses_Aboard, 
        Parents_Children_Aboard, Fare)
        VALUES """ + str(people[1:]) + ";"
    pg_curs.execute(insert_data)

In [21]:
pg_curs.close()
pg_conn.commit()

In [22]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host) #reopened connection
pg_curs = pg_conn.cursor() #reopened the cursor

In [23]:
pg_curs.execute('SELECT * FROM titanic;')
new_titanic = pg_curs.fetchall()

In [24]:
original_titanic[0:3]

[(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25),
 (1,
  1,
  1,
  'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'female',
  38.0,
  1,
  0,
  71.2833),
 (2, 1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 0, 0, 7.925)]

In [25]:
new_titanic[0:3]

[(0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25),
 (1,
  1,
  'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'female',
  38.0,
  1,
  0,
  71.2833),
 (1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 0, 0, 7.925)]

In [36]:
# somethings match exactly, and some things don't
for data, pg_data in zip(original_titanic, new_titanic):
    if data[1:] == pg_data:
        print('hurray')
    else:
        print('cry')

hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
cry
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
cry
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurray
hurra